# Initialization

In [1]:
import pandas as pd
from bangla_character import bangla
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
import re # re is used for regular expressions
import os # os is used for operating system related functions
import torch # torch is used for building deep learning models
from normalizer import normalize

# Data pre-processing

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

## dealing with nulls

In [3]:
test_df.isnull().sum()

Id         0
summary    0
dtype: int64

In [4]:
train_df.isnull().sum()

summary    1
genre      0
Label      0
dtype: int64

In [5]:
train_df.loc[pd.isna(train_df["summary"]), :].index

Int64Index([1317], dtype='int64')

In [6]:
train_df=train_df.dropna().reset_index(drop=True)

## dealing with english

In [7]:
filtered_train = train_df[lambda x: x["summary"].str.contains("[A-Za-z0-9]")]

In [8]:
filtered_train

,summary,genre,Label
0,#BookReview\r\n\r\nবইঃ চাঁদের পাহাড়\r\nধরনঃ অ...,Childrens-Book,2
6,#রকমারি_বইপোকা_রিভিউ_প্রতিযোগিতা বইঃ বোতল ভূত ...,Childrens-Book,2
12,#রকমারি_বইপোকা_রিভিউ_প্রতিযোগিতা বই: দ্য বুক অ...,Childrens-Book,2
15,"বুক রিভিউ- ""রিচার্জ Your ডাউন ব্যাটারি"" -ঝংকার...",Motivational,6
17,বনি গল্পের শুরু হয় আমেরিকা প্রবাসী এক বাঙালি দ...,Science-Fiction,4
...,...,...,...
3847,১৯৭১ সালের ১৬ ডিসেম্বর। আমরা থাকি এলিফ্যান্ট র...,War,5
3854,বইঃ গ্রামের নাম কাঁকনডুবি\r\nলেখকঃ মুহম্মদ জাফ...,Childrens-Book,2
3861,#রকমারি_বইপোকা_রিভিউ_প্রতিযোগিতা\r\nবই:টুকি ও ...,Science-Fiction,4
3862,Himu... himu... himu\r\nWhat can I really say ...,Fiction,0


In [9]:
for i in range(221):
    str(bangla.character(i+1))

অ
আ
ই
ঈ
উ
ঊ
ঋ
এ
ঐ
ও
ঔ
ক
খ
গ
ঘ
ঙ
চ
ছ
জ
ঝ
ঞ
ট
ঠ
ড
ঢ
ণ
ত
থ
দ
ধ
ন
প
ফ
ব
ভ
ম
য
র
ল
শ
ষ
স
হ
ড়
ঢ়
য়
ৎ
ং
ঃ
ঁ
প্র
ঙ্গ
ক্ষ
ত্র
ন্দ
চ্ছ
ন্ত
ন্দ্র
স্ত
ন্তু
গ্র
স্থ
স্ট
ম্ব
স্ব
ত্ত 
ক্ত 
ন্ট 
ল্প 
ষ্ট 
ন্দ্র 
ক্র 
ন্ন 
দ্ধ 
ন্ধ 
ঙ্ক 
ন্ড 
ফ্র 
ম্প 
স্ক 
জ্ঞ 
ক্ট 
শ্চ 
ট্র 
ত্ব 
ল্ল 
ব্র 
ঞ্চ 
ন্ড 
ক্স 
শ্র 
দ্র 
স্প 
ঞ্জ 
ন্স 
ম্ভ 
শ্ব 
ব্দ 
শ্ন 
প্প 
ব্ল 
প্ত 
ক্ল 
ষ্ট্র 
দ্ব 
ট্ট 
গ্ল 
ল্ট 
ষ্ঠ 
স্ত্র 
প্ল 
চ্চ 
স্ম 
দ্দ 
গ্ন 
জ্ব 
ষ্ক 
ত্ম 
ড্র 
ম্ম 
ন্ট 
ম্প্র 
প্ন 
ন্ম 
স্ফ 
ল্‌দ 
ত্ত্ব 
জ্জ 
ক্ষ্ম 
ষ্ণ 
ন্ব 
ক্ক 
ন্থ 
ড্ড 
ব্ব 
ন্ট্র 
ন্ঠ 
প্ট 
স্‌ত্ত 
ধ্ব 
হ্ন 
ভ্র 
ল্ক 
স্ল 
হ্ন 
ত্ম 
ষ্ক্র 
ঘ্র 
দ্ভ 
শ্ল 
ব্ধ 
ষ্ম 
স্ক্র 
ড়্গ 
জ্জ্ব 
শ্ম 
দ্ম 
ক্ব 
ম্র 
গ্ধ 
ব্জ 
স্ন 
ন্দ্ব 
ক্ষ 
ঙ্ঘ 
খ্র 
ণ্‌থ 
ল্ব 
ম্ন 
ঘ্ন 
গ্‌গ 
ক্ষ্‌ন 
গ্রু 
চ্ছ্ব 
ণ্ন 
ল্ম 
স্র 
ম্ল 
ষ্প্র 
ঞ্ঝ 
স্প্র 
ম্ভ্র 
ষ্প 
ঙ্খ 
জ্র 
গ্ব 
থ্ব 
ন্ব 
হ্ব 
দ্দ্ব 
দ্ম 
ধ্র 
হ্ল 
গ্ম 
ল্গ 
স্থ 
থ্র 
ন্ধ্র 
ফ্ল 
ঙ্ক্ষ 
ন্ম 
ঞ্ছ 
ম্ফ 
হ্র 
প্রু 
ত্রু 
ভ্ল 
শ্রু 
দ্রু 
ঙ্ম 
ক্ম 
চ্ঞ 
দ্গ 
ন্ড্র 
ট্ব 
প্স 
ল্ড 
ষ্ক 
শ্চ 
জ্ঝ 
স্ট্র


In [10]:
bangla_alnum = '[০১২৩৪৫৬৭৮৯অআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহড়ঢ়য়ৎংঃঁ]'

# texts with bangla and english mixed
filtered_train_2 = filtered_train[lambda x: x["summary"].str.contains(bangla_alnum)]

In [11]:
filtered_train_2

,summary,genre,Label
0,#BookReview\r\n\r\nবইঃ চাঁদের পাহাড়\r\nধরনঃ অ...,Childrens-Book,2
6,#রকমারি_বইপোকা_রিভিউ_প্রতিযোগিতা বইঃ বোতল ভূত ...,Childrens-Book,2
12,#রকমারি_বইপোকা_রিভিউ_প্রতিযোগিতা বই: দ্য বুক অ...,Childrens-Book,2
15,"বুক রিভিউ- ""রিচার্জ Your ডাউন ব্যাটারি"" -ঝংকার...",Motivational,6
17,বনি গল্পের শুরু হয় আমেরিকা প্রবাসী এক বাঙালি দ...,Science-Fiction,4
...,...,...,...
3845,#বই_পর্যালোচনা নাম : থ্রি এ এম (3:00 AM) লেখক ...,Thriller,1
3847,১৯৭১ সালের ১৬ ডিসেম্বর। আমরা থাকি এলিফ্যান্ট র...,War,5
3854,বইঃ গ্রামের নাম কাঁকনডুবি\r\nলেখকঃ মুহম্মদ জাফ...,Childrens-Book,2
3861,#রকমারি_বইপোকা_রিভিউ_প্রতিযোগিতা\r\nবই:টুকি ও ...,Science-Fiction,4


In [12]:
# texts with only english alphabets
train_eng = filtered_train.loc[filtered_train.index.difference(filtered_train_2.index), ]

In [13]:
train_eng

,summary,genre,Label
24,good really,Fiction,0
37,it is very good,Fiction,0
38,When this book will available in your stock,Political,3
41,this book is anisul haque's first novel and in...,Fiction,0
62,my book,Political,3
...,...,...,...
3782,best book ever,Political,3
3783,It is a Great Book of that time Political Parties,Political,3
3803,Khub bhalo boi .............,Fiction,0
3816,A brief journey into Pakistan period...,Political,3


In [14]:
# texts without only english alphabets
train_not_eng = train_df.loc[train_df.index.difference(train_eng.index), ]

In [15]:
train_not_eng

,summary,genre,Label
0,#BookReview\r\n\r\nবইঃ চাঁদের পাহাড়\r\nধরনঃ অ...,Childrens-Book,2
1,বেশির ভাগ বই ই আমার ভাললাগে। কিন্তু এই টাইপের ...,Fiction,0
2,মিসির আলি এখন তার ছাত্র ফারুকের শ্বশুর বাড়ি কই...,Fiction,0
3,ছোটজাতের ছোটজাত হাসিম। পিতৃপ্রদত্ত নামখানা একর...,Fiction,0
4,ফালতু একটা বই। পয়সা নষ্ট। বাজে লেখনী। একই জিনি...,Political,3
...,...,...,...
3881,"বিশ্ববিদ্যালয়ের অভ্যন্তরীণ কিছু ঘটনা, কিছু মা...",Fiction,0
3882,২০০৫ সালে সপ্তম শ্রেণীতে উঠে সাপ্তাহিক বই পড়া ...,Science-Fiction,4
3883,আবহমান গ্রাম বাংলার একটি একান্নবর্তী পরিবারের ...,Fiction,0
3884,আমার মত যারা ৯০'র দশকে জন্ম তারা আমরা অনেকেই ১...,Political,3


In [16]:
# Remove English alphanumeric values
alpha_pat = "[a-zA-z0-9]"

train_not_eng["summary"] = train_not_eng["summary"].str.replace(alpha_pat, "", regex=True)

In [17]:
train_not_eng['Label'].value_counts()

0    1250
1     693
2     493
4     408
3     375
5     296
6     196
Name: Label, dtype: int64

##  Data Normalization

In [18]:
train_not_eng['summary'] = pd.DataFrame(train_not_eng['summary'].map(lambda x: normalize(x)))

In [19]:
train_not_eng = train_not_eng.reset_index()
train_not_eng = train_not_eng.drop(columns = ['genre', 'index'])

In [20]:
train_not_eng

,summary,Label
0,# বইঃ চাঁদের পাহাড় ধরনঃ অ্যাডভেঞ্চার লেখকঃ বি...,2
1,বেশির ভাগ বই ই আমার ভাললাগে। কিন্তু এই টাইপের ...,0
2,মিসির আলি এখন তার ছাত্র ফারুকের শ্বশুর বাড়ি ক...,0
3,ছোটজাতের ছোটজাত হাসিম। পিতৃপ্রদত্ত নামখানা একর...,0
4,ফালতু একটা বই। পয়সা নষ্ট। বাজে লেখনী। একই জিন...,3
...,...,...
3706,"বিশ্ববিদ্যালয়ের অভ্যন্তরীণ কিছু ঘটনা, কিছু মা...",0
3707,২০০৫ সালে সপ্তম শ্রেণীতে উঠে সাপ্তাহিক বই পড়া...,4
3708,আবহমান গ্রাম বাংলার একটি একান্নবর্তী পরিবারের ...,0
3709,আমার মত যারা ৯০'র দশকে জন্ম তারা আমরা অনেকেই ১...,3


## Folding

In [21]:
SEED = 42
from sklearn.model_selection import StratifiedKFold  # Import package

skf = StratifiedKFold(n_splits=4 , shuffle=True, random_state=SEED)  # Initialize K-Fold

#df = df.reset_index(drop=True)  # Reset dataframe index

#train_not_eng['stratify'] = train_not_eng.Label.astype(str)

train_not_eng["fold"] = -1  # New 'fold' column

# Assign folds using StratifiedKFold
for fold, (train_idx, val_idx) in enumerate(skf.split(train_not_eng, train_not_eng['Label'])):
    train_not_eng.loc[val_idx, 'fold'] = fold

In [22]:
train_not_eng.groupby(["fold",'Label']).size()

fold  Label
0     0        312
      1        173
      2        124
      3         94
      4        102
      5         74
      6         49
1     0        313
      1        173
      2        123
      3         94
      4        102
      5         74
      6         49
2     0        313
      1        174
      2        123
      3         93
      4        102
      5         74
      6         49
3     0        312
      1        173
      2        123
      3         94
      4        102
      5         74
      6         49
dtype: int64

In [23]:
train_df = train_not_eng.loc[lambda x: x['fold'] != 3]
val_df = train_not_eng.loc[lambda x: x['fold'] == 3]

In [24]:
train_df

,summary,Label,fold
2,মিসির আলি এখন তার ছাত্র ফারুকের শ্বশুর বাড়ি ক...,0,0
3,ছোটজাতের ছোটজাত হাসিম। পিতৃপ্রদত্ত নামখানা একর...,0,1
4,ফালতু একটা বই। পয়সা নষ্ট। বাজে লেখনী। একই জিন...,3,0
5,#রকমারিবইপোকারিভিউপ্রতিযোগিতা। বইয়ের নামঃ আর ...,0,2
6,#রকমারিবইপোকারিভিউপ্রতিযোগিতা বইঃ বোতল ভূত লেখ...,2,0
...,...,...,...
3706,"বিশ্ববিদ্যালয়ের অভ্যন্তরীণ কিছু ঘটনা, কিছু মা...",0,2
3707,২০০৫ সালে সপ্তম শ্রেণীতে উঠে সাপ্তাহিক বই পড়া...,4,0
3708,আবহমান গ্রাম বাংলার একটি একান্নবর্তী পরিবারের ...,0,1
3709,আমার মত যারা ৯০'র দশকে জন্ম তারা আমরা অনেকেই ১...,3,0


In [25]:
train_df = train_df.reset_index().drop(columns = ['fold', 'index'])
val_df = val_df.reset_index().drop(columns = ['fold', 'index'])

In [26]:
train_df = train_df.rename(columns={"summary": "sentence1", "Label": "label"})
val_df = val_df.rename(columns={"summary": "sentence1", "Label": "label"})

In [27]:
train_df

,sentence1,label
0,মিসির আলি এখন তার ছাত্র ফারুকের শ্বশুর বাড়ি ক...,0
1,ছোটজাতের ছোটজাত হাসিম। পিতৃপ্রদত্ত নামখানা একর...,0
2,ফালতু একটা বই। পয়সা নষ্ট। বাজে লেখনী। একই জিন...,3
3,#রকমারিবইপোকারিভিউপ্রতিযোগিতা। বইয়ের নামঃ আর ...,0
4,#রকমারিবইপোকারিভিউপ্রতিযোগিতা বইঃ বোতল ভূত লেখ...,2
...,...,...
2779,"বিশ্ববিদ্যালয়ের অভ্যন্তরীণ কিছু ঘটনা, কিছু মা...",0
2780,২০০৫ সালে সপ্তম শ্রেণীতে উঠে সাপ্তাহিক বই পড়া...,4
2781,আবহমান গ্রাম বাংলার একটি একান্নবর্তী পরিবারের ...,0
2782,আমার মত যারা ৯০'র দশকে জন্ম তারা আমরা অনেকেই ১...,3


In [28]:
train_df.to_json("train.jsonl", orient='records', lines=True)
val_df.to_json("validation.jsonl", orient='records', lines=True)

# English

In [29]:
train_eng

,summary,genre,Label
24,good really,Fiction,0
37,it is very good,Fiction,0
38,When this book will available in your stock,Political,3
41,this book is anisul haque's first novel and in...,Fiction,0
62,my book,Political,3
...,...,...,...
3782,best book ever,Political,3
3783,It is a Great Book of that time Political Parties,Political,3
3803,Khub bhalo boi .............,Fiction,0
3816,A brief journey into Pakistan period...,Political,3


In [30]:
train_eng['Label'].value_counts()

0    95
3    64
1     9
2     4
4     2
5     1
Name: Label, dtype: int64

In [31]:
train_eng = train_eng.reset_index()
train_eng = train_eng.drop(columns = ['genre', 'index'])

In [32]:
train_eng

,summary,Label
0,good really,0
1,it is very good,0
2,When this book will available in your stock,3
3,this book is anisul haque's first novel and in...,0
4,my book,3
...,...,...
170,best book ever,3
171,It is a Great Book of that time Political Parties,3
172,Khub bhalo boi .............,0
173,A brief journey into Pakistan period...,3


In [33]:
SEED = 42
from sklearn.model_selection import StratifiedKFold  # Import package

skf = StratifiedKFold(n_splits=4 , shuffle=True, random_state=SEED)  # Initialize K-Fold

#df = df.reset_index(drop=True)  # Reset dataframe index

#train_not_eng['stratify'] = train_not_eng.Label.astype(str)

train_eng["fold"] = -1  # New 'fold' column

# Assign folds using StratifiedKFold
for fold, (train_idx, val_idx) in enumerate(skf.split(train_eng, train_eng['Label'])):
    train_eng.loc[val_idx, 'fold'] = fold

C:\Users\Niaz Mohaiman Abtahi\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


In [34]:
train_eng.groupby(["fold",'Label']).size()

fold  Label
0     0        24
      1         2
      2         1
      3        16
      4         1
1     0        24
      1         2
      2         1
      3        16
      4         1
2     0        24
      1         2
      2         1
      3        16
      5         1
3     0        23
      1         3
      2         1
      3        16
dtype: int64

In [35]:
train_eng_df = train_eng.loc[lambda x: x['fold'] != 0]
val_eng_df = train_eng.loc[lambda x: x['fold'] == 0]

In [36]:
train_eng_df = train_eng_df.reset_index().drop(columns = ['fold', 'index'])
val_eng_df = val_eng_df.reset_index().drop(columns = ['fold', 'index'])

In [37]:
train_eng_df = train_eng_df.rename(columns={"summary": "sentence1", "Label": "label"})
val_eng_df = val_eng_df.rename(columns={"summary": "sentence1", "Label": "label"})

In [38]:
train_eng_df

,sentence1,label
0,good really,0
1,When this book will available in your stock,3
2,this book is anisul haque's first novel and in...,0
3,my book,3
4,"Ami banglali-der bolchi, Ai boi poro ... Nijer...",3
...,...,...
126,Almost all the stories written by Dr. Jafor Si...,2
127,best book ever,3
128,Khub bhalo boi .............,0
129,A brief journey into Pakistan period...,3


In [39]:
train_eng_df.to_json("train_eng.jsonl", orient='records', lines=True)
val_eng_df.to_json("validation_eng.jsonl", orient='records', lines=True)